# Fantasy Sidelines 
### Code to Scrape, Clean, Save, and Analyze NFL injury, player stats, team stats, and player snaps data.

## Libraries used in code

In [1]:
import dash, requests, os, time, dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import datetime as dt
import numpy as np
from datetime import date, timedelta
from bs4 import BeautifulSoup
from dash.dependencies import Input, Output
from dotenv import load_dotenv
from dash_table.Format import Format, Scheme, Trim
from jupyter_dash import JupyterDash
load_dotenv()
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

## Section 1 Scrape Data

### 1.1 - Team Weekly Stats Scraped From www.stathead.com

In [ ]:
def team_data_scrape(Season,url):
    # scrape weekly team stats from www.stathead.com
    start = time.time()
    # page to start scrape at
    page = 0
    # login payload information pulled from a .env file
    stat_login_url = "https://stathead.com/users/login.cgi"
    stat_user_name = os.environ.get('statheadusername')
    stat_password = os.environ.get('statheadpassword')
    stat_payload = {
        'username': stat_user_name,
        'password': stat_password
    }
    
    # lots of team data, could not fit into one pull from stathead, needed to use 2 different url's, this allows the function to iterate through both
    if url == 1:
        stat_url = 'https://stathead.com/football/tgl_finder.cgi?request=1&temperature_gtlt=lt&game_num_max=99&week_num_max=99&order_by=points&match=game&year_max={Season}&order_by_asc=0&week_num_min=0&game_type=E&game_num_min=0&year_min={Season}&cstat[1]=all_td_team&ccomp[1]=gt&cval[1]=0&cstat[2]=third_down_att&ccomp[2]=gt&cval[2]=0&cstat[3]=vegas_line&ccomp[3]=gt&cval[3]=-50&cstat[4]=penalties&ccomp[4]=gt&cval[4]=0&cstat[5]=rush_att&ccomp[5]=gt&cval[5]=0&cstat[6]=tot_yds&ccomp[6]=gt&cval[6]=0&cstat[7]=first_down&ccomp[7]=gt&cval[7]=0&cstat[8]=punt&ccomp[8]=gt&cval[8]=0&cstat[9]=pass_cmp&ccomp[9]=gt&cval[9]=0&offset={page}'
    elif url == 2:
        stat_url = 'https://stathead.com/football/tgl_finder.cgi?request=1&temperature_gtlt=lt&game_num_max=99&week_num_max=99&order_by=all_td_opp&match=game&year_max={Season}&order_by_asc=0&week_num_min=0&game_type=R&game_num_min=0&year_min={Season}&cstat[1]=tot_yds_opp&ccomp[1]=gt&cval[1]=0&cstat[2]=rush_yds_diff&ccomp[2]=gt&cval[2]=-500&cstat[3]=score_diff_thru_1&ccomp[3]=gt&cval[3]=-500&cstat[4]=rush_att_opp&ccomp[4]=gt&cval[4]=0&cstat[5]=kick_ret_td_tgl&ccomp[5]=gt&cval[5]=0&cstat[6]=pass_cmp_opp&ccomp[6]=gt&cval[6]=0&cstat[7]=first_down_opp&ccomp[7]=gt&cval[7]=0&cstat[8]=score_diff_1_qtr&ccomp[8]=gt&cval[8]=-500&cstat[9]=third_down_att_opp&ccomp[9]=gt&cval[9]=0&offset={page}'
    elif url != 1 or 2:
        print("Please select 1 or 2.")
    
    # open logged in session for scraping
    with requests.Session() as session:

        s = session.post(stat_login_url, data=stat_payload)
        
        # beginning the scrape and stopping the scrape when page number reaches 100k
        try:

            while page < 100000:
                
                # pulling the website and scraping it
                website = session.get(stat_url.format(Season=Season,page=page)).text
                soup = BeautifulSoup(website, 'html')
                table = soup.find('table', attrs={'class': 'sortable', 'id': 'results'})
                
                # pull headers and rows out of the data
                table_headers = [header.text for header in table.find('thead').find_all('th')]
                table_rows = table.find_all('tr')
                
                # final location for complete data
                final_data = []
                
                # create row for each line of data in table
                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text for tr in td]
                    final_data.append(row)
                
                # create the dataframe in panadas excluding the blank row and matching headers with the rows
                df = pd.DataFrame(final_data[1:], columns=table_headers[12:])
                
                # writting dataframe to csv, continuous appending just incase the function fails, data will be saved
                if url == 1:
                    df.to_csv(f'../data/raw-data/nfl-team-data-{Season}-1-raw.csv',mode='a',index=False)
                else:
                    df.to_csv(f'../data/raw-data/nfl-team-data-{Season}-2-raw.csv',mode='a',index=False)
                
                # progress through the websites
                page += 100
            
        except:
            # notifying the scrape has completed.
            end = time.time()
            print(f'Done: Team {Season}, {url}, {page}',f'Time to complete: {end-start}',sep='\n')

### 1.2 - Player Weekly Snap Data Scraped From www.pro-football-reference.com

In [ ]:
def player_snap_scrape(Season_Start, Season_End, p_database):
    
    start = time.time()
    
    p_database = p_database.lower()

    if p_database == "y":

        abcd = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
        player_url = "https://www.pro-football-reference.com/players/{abcd}/"
        players = []
        for c in range(0,len(abcd)):
            res = requests.get(player_url.format(abcd=abcd[c]))
            soup = BeautifulSoup(res.content, 'html.parser')
            data = soup.find_all('p')
            try:
                for i in data:
                    a = i.find('a')['href']
                    pl = i.get_text()[:(i.get_text().find("(")-1)]
                    po = i.get_text()[(i.get_text().find("(")+1):i.get_text().find(")")]
                    fy = i.get_text()[(i.get_text().rfind("-")-4):(i.get_text().rfind("-"))]
                    ly = i.get_text()[(i.get_text().rfind("-")+1):(i.get_text().rfind("-")+5)]
                    p = {"Address": a[:-4], 
                         "Player": pl, 
                         "Position": po, 
                         "First_Year":  fy, 
                         "Last_Year": ly}
                    players.append(p)
            except:
                pass
        players_df = pd.DataFrame().from_dict(players)
        players_df = players_df[players_df.Last_Year != "Ever"]
        players_df['Player'] = players_df['Player'].str.replace('+','')
        players_df['Last_Year'] = players_df.Last_Year.astype('int32')
        players_df = players_df[players_df.Last_Year >= Season_Start]
        players_df.to_csv('../data/database-players.csv',index=False)

    players_df = pd.read_csv('../data/database-players.csv')

    players_address_list = players_df.Address.tolist()

    snap_counts_df = pd.DataFrame()

    nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')

    for player_address in players_address_list:

            try:

                df = pd.DataFrame()
                df['Player_Address'] = ''
                df['Date'] = ''

                for s in range(Season_Start, Season_End+1):

                    try:
                        snap_counts_url = "https://www.pro-football-reference.com{player_address}/fantasy/{s}"

                        # opening the webpage and storing data into lists then to dataframe
                        res = requests.get(snap_counts_url.format(player_address=player_address,s=s))

                        soup = BeautifulSoup(res.content, 'lxml')

                        table = soup.find('table', {'id': 'player_fantasy'})
                        table_headers = [header.text for header in table.find('thead').find_all('th')]
                        table_rows = table.find_all('tr')

                        final_data = []
                        snap_counts_df = pd.DataFrame()

                        for tr in table_rows:
                            td = tr.find_all('td')
                            row = [tr.text for tr in td]
                            final_data.append(row)     

                        snap_counts_df = pd.DataFrame(final_data[:-1], columns=table_headers[11:])
                        snap_counts_df.columns = ['G#', 'Date', 'Team', 'Home_Away', 'Opp', 'Result', 
                                                  'Pos', 'Rec_Tgt_In20', 'Rec_Rec_In20', 'Rec_Yds_In20', 
                                                  'Rec_TD_In20', 'Off_Snaps', 'Off_Snaps%', 'Def_Snaps', 
                                                  'Def_Snaps%', 'ST_Snaps', 'ST_Snaps%', 'Fntsy_FanPts', 
                                                  'Fntsy_DKPt', 'Fntsy_FDPt']
                        snap_counts_df['Player_Address'] = player_address
                        snap_counts_df = snap_counts_df[snap_counts_df.Date >= "2016-08-01"]
                        snap_counts_df['Date'] = pd.to_datetime(snap_counts_df['Date'],errors='coerce',format='%Y-%m-%d')
                        snap_counts_df['Season'] = s
                        nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
                        # create function to calculate what nfl week the date occured
                        def pre_thu(d):
                            days_behind = 3 - d.weekday()
                            if days_behind > 0:
                                days_behind -= 7
                            return d + dt.timedelta(days_behind)
                        snap_counts_df['week_start_nfl'] = snap_counts_df['Date'].apply(pre_thu)
                        nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
                        # merge injury data with the nfl week data to pull the weeks into current datagrame
                        snap_counts_df = pd.merge(left=snap_counts_df,right=nfl_weeks,how='left',left_on='week_start_nfl',right_on='Start Date')
                        snap_counts_df = snap_counts_df[['Player_Address', 'Pos', 'Team', 
                                                         'Home_Away', 'Opp', 'Result', 
                                                         'Date', 'Season', 'Week', 'G#', 
                                                         'Off_Snaps', 'Off_Snaps%', 
                                                         'Def_Snaps', 'Def_Snaps%', 
                                                         'ST_Snaps', 'ST_Snaps%']]
                        snap_counts_df.replace({'Home_Away': {'@': 'Away', '': 'Home'}},regex=True,inplace=True)
                        snap_counts_df['Off_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['Off_Snaps%'].values))
                        snap_counts_df['Def_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['Def_Snaps%'].values))
                        snap_counts_df['ST_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['ST_Snaps%'].values))
                        snap_counts_df.to_csv("../data/raw-data/snap-counts-raw.csv", mode="a", index=False)

                    except:
                        snap_counts_url = "https://www.pro-football-reference.com{player_address}/fantasy/{s}"

                        # opening the webpage and storing data into lists then to dataframe
                        res = requests.get(snap_counts_url.format(player_address=player_address,s=s))

                        soup = BeautifulSoup(res.content, 'lxml')

                        table = soup.find('table', {'id': 'player_fantasy'})
                        table_headers = [header.text for header in table.find('thead').find_all('th')]
                        table_rows = table.find_all('tr')

                        final_data = []
                        snap_counts_df = pd.DataFrame()

                        for tr in table_rows:
                            td = tr.find_all('td')
                            row = [tr.text for tr in td]
                            final_data.append(row)

                        snap_counts_df = pd.DataFrame(final_data[:-1], columns=table_headers[9:])
                        snap_counts_df.columns = ['G#', 'Date', 'Team', 'Home_Away', 'Opp', 'Result', 
                                                  'Pos', 'Off_Snaps', 'Off_Snaps%', 'Def_Snaps', 
                                                  'Def_Snaps%', 'ST_Snaps', 'ST_Snaps%', 'Fntsy_FanPts', 
                                                  'Fntsy_DKPt', 'Fntsy_FDPt']
                        snap_counts_df['Player_Address'] = player_address
                        snap_counts_df = snap_counts_df[snap_counts_df.Date >= "2016-08-01"]
                        snap_counts_df['Date'] = pd.to_datetime(snap_counts_df['Date'],errors='coerce',format='%Y-%m-%d')
                        snap_counts_df['Season'] = s
                        nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
                        # create function to calculate what nfl week the date occured
                        def pre_thu(d):
                            days_behind = 3 - d.weekday()
                            if days_behind > 0:
                                days_behind -= 7
                            return d + dt.timedelta(days_behind)
                        snap_counts_df['week_start_nfl'] = snap_counts_df['Date'].apply(pre_thu)
                        nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
                        # merge injury data with the nfl week data to pull the weeks into current datagrame
                        snap_counts_df = pd.merge(left=snap_counts_df,right=nfl_weeks,how='left',left_on='week_start_nfl',right_on='Start Date')
                        snap_counts_df = snap_counts_df[['Player_Address', 'Pos', 'Team', 
                                                         'Home_Away', 'Opp', 'Result', 
                                                         'Date', 'Season', 'Week', 'G#', 
                                                         'Off_Snaps', 'Off_Snaps%', 
                                                         'Def_Snaps', 'Def_Snaps%', 
                                                         'ST_Snaps', 'ST_Snaps%']]
                        snap_counts_df.replace({'Home_Away': {'@': 'Away', '': 'Home'}},regex=True,inplace=True)
                        snap_counts_df['Off_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['Off_Snaps%'].values))
                        snap_counts_df['Def_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['Def_Snaps%'].values))
                        snap_counts_df['ST_Snaps%'] = list(map(lambda x: x[:-1], snap_counts_df['ST_Snaps%'].values))
                        snap_counts_df.to_csv("../data/raw-data/snap-counts-raw.csv", mode="a", index=False)
            except:
                pass
    end = time.time()
    print(f'Done: Player Snaps from {Season_Start} to {Season_End}', f'Time to complete: {end-start}',sep='\n')

### 1.3 - NFL Weekly Injury Reports Scraped From www.pro-football-reference.com

In [ ]:
def injury_reports_scrape(Season):

    # scrape weekly nfl injury reports
    start = time.time()
    # list of team abbreviations from pro-football-reference for url purposes
    teams = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den', 'det', 'gnb','htx','clt','jax','kan',
             'sdg','ram','mia','min','nor','nwe','nyg','nyj','rai','phi','pit','sea','sfo','tam','oti','was']

    ENDPOINT = 'https://www.pro-football-reference.com/teams/{team}/{Season}_injuries.htm'
    
    # creating final place to store data
    final_df = pd.DataFrame()

    # scrape pages for all teams
    for team in teams:
        
        # open webpage and scrape contents into lists, then to a dataframe
        res = requests.get(ENDPOINT.format(Season=Season,team=team))

        soup = BeautifulSoup(res.content, 'lxml')

        table = soup.find('table', attrs={'class': 'sortable', 'id': 'team_injuries'})
        table_rows = table.find_all('tr')

        final_data = []
        for tr in table_rows:
            td = tr.find_all(['th','td'])
            row = [tr['data-tip'] if tr.has_attr("data-tip") else tr.text for tr in td]
            final_data.append(row)

        dfdata = final_data[1:]
        data_body = [[dfdata[j][i] for j in range(len(dfdata))] for i in range(len(dfdata[0]))]

        df = pd.DataFrame(data_body,final_data[0]).T
        # adding team and season columns for identification
        df.insert(loc=1,column='Team',value=team)
        df.insert(loc=2,column='Season',value=Season)

        # combine current data with final dataframe
        final_df = pd.concat([final_df, df])
    
    # rename column
    final_df.rename(columns={'PlayerÂ ':'Player'},inplace=True)
    
    # write final data to csv file
    final_df.to_csv(f'../data/raw-data/nfl-injury-report-{Season}-raw.csv',index=False)
    end = time.time()
    print(f'Done: Injury Reports {Season}',f'Time to complete: {end-start}',sep='\n')

### 1.4 - Player Weekly Stats Scraped From www.stathead.com

In [ ]:
def player_stats_scape(Season):
    
    # scrape weekly player stats from statehead.com
    start = time.time()
    # page/location for monitoring progress and continuing through pages
    page = 0
    location = 2000
    
    # stathead login info pulled from .env file
    stat_login_url = "https://stathead.com/users/login.cgi"
    stat_user_name = os.environ.get('statheadusername')
    stat_password = os.environ.get('statheadpassword')
    stat_payload = {
        'username': stat_user_name,
        'password': stat_password
    }
    stat_url = "https://stathead.com/football/pgl_finder.cgi?request=1&game_num_max=99&week_num_max=99&order_by=all_td&season_start=1&qb_gwd=0&order_by_asc=0&qb_comeback=0&week_num_min=0&game_num_min=0&year_min={Season}&match=game&year_max={Season}&season_end=-1&age_min=0&game_type=R&age_max=99&positions[]=qb&positions[]=rb&positions[]=wr&positions[]=te&positions[]=e&positions[]=t&positions[]=g&positions[]=c&positions[]=ol&positions[]=dt&positions[]=de&positions[]=dl&positions[]=ilb&positions[]=olb&positions[]=lb&positions[]=cb&positions[]=s&positions[]=db&positions[]=k&positions[]=p&cstat[1]=punt_ret&ccomp[1]=gt&cval[1]=0&cstat[2]=sacks&ccomp[2]=gt&cval[2]=0&cstat[3]=fumbles&ccomp[3]=gt&cval[3]=0&cstat[4]=rush_att&ccomp[4]=gt&cval[4]=0&cstat[5]=pass_defended&ccomp[5]=gt&cval[5]=0&cstat[6]=pass_cmp&ccomp[6]=gt&cval[6]=0&cstat[7]=targets&ccomp[7]=gt&cval[7]=0&cstat[8]=kick_ret&ccomp[8]=gt&cval[8]=0&offset={page}"

    # logging into session to begin scrape
    with requests.Session() as session:

        s = session.post(stat_login_url, data=stat_payload)
        
        # begin scrape, once it fails, stop scrape
        try:

            # scrape webpages up to 100k contents
            while page < 100000:

                # opening webpage and storing data into list then dataframe
                website = session.get(stat_url.format(Season=Season,page=page)).text
                soup = BeautifulSoup(website, 'html')
                table = soup.find('table', attrs={'class': 'sortable', 'id': 'results'})

                table_headers = [header.text for header in table.find('thead').find_all('th')]
                table_rows = table.find_all('tr')

                final_data = []

                for tr in table_rows:
                    td = tr.find_all('td')
                    row = [tr.text for tr in td]
                    final_data.append(row)

                df = pd.DataFrame(final_data[1:], columns=table_headers[11:])
                df.rename(columns={'Year':'Season'},inplace=True)
                
                # appendings csv with data from current dataframe to prevent loss if code fails or connection drops
                df.to_csv(f'../data/raw-data/player-stats-{Season}-raw.csv',mode='a',index=False)

                # continue through webpages and update where the location of the scrape is
                if page > location:
                    print('Player Stats on page:',(page-100))
                    location += 2000

                page += 100
                
        except:
            # end of the scrape notificaiton
            end = time.time()
            print(f'Done: Stats {Season}, {page}',f'Time to complete: {end-start}',sep='\n')

## Section 2 - Clean data

### 2.1 - Clean Team Weekly Stats

In [ ]:
def team_data_clean(Season):
    # begin cleaning team data
    start = time.time()
    # opening both csv files into dataframes for cleaning and combining
    team1 = pd.read_csv(f'../data/raw-data/nfl-team-data-{Season}-1-raw.csv')
    team2 = pd.read_csv(f'../data/raw-data/nfl-team-data-{Season}-2-raw.csv')
    # drop all blank rows
    team1.dropna(thresh=10,inplace=True)
    team2.dropna(thresh=10,inplace=True)
    # drop useless columns
    team1.drop('LTime',axis=1,inplace=True)
    team2.drop(['LTime'],axis=1,inplace=True)
    # rename columns
    team1.rename(columns={'Tm':'Team','Unnamed: 5':'Away_Home','PF':'Points_For','PA':'Points_Against','PC':'Points_Comb',\
                         'vs. Line':'Vs_Line','Cmp':'TPass_Cmp','Att':'TPass_Att','Cmp%':'TPass_Cmp%','Yds':'TPass_Yds',\
                          'TD':'TPass_TD','Int':'TPass_Int','Sk':'TSack','Yds.1':'TSack_Yds','Rate':'TQB_Rate',\
                          'Att.1':'TRush_Att','Yds.2':'TRush_Yds','Y/A':'TRush_Y/A','TD.1':'TRush_TD','Tot':'TTot_Yds',\
                          'Ply':'TO_Play#','Y/P':'TO_Y/P','DPly':'TD_Play#','DY/P':'TD_Y/P','TO':'TTot_TO','ToP':'TO_ToP',\
                          'Time.1':'TGame_Dur','Yds.3':'TPen_Yds','OppPen':'TOpp_Pen','OppYds':'TOpp_Pen_Yds',\
                          'CombPen':'TComb_Pen','CombPenYds':'TComb_Pen_Yds','1stD':'T1st_Downs','Rsh':'T1st_by_Rsh',\
                          'Pass':'T1st_by_Pass','Pen.1':'T1st_by_Pen','3DAtt':'T3rd_Down_Att','3DConv':'T3rd_Down_Conv',\
                          '3D%':'T3rd_Down%','4DAtt':'T4th_Down_Att','4DConv':'T4th_Down_Conv','4D%':'T4th_Down%',\
                          'TD.2':'TTot_TD','XPA':'TXP_Att','XPM':'TXP_Made','FGA':'TFG_Att','FGM':'TFG_Made','2PA':'T2Pt_Att',\
                          '2PM':'T2Pt_Made','Sfty':'TSfty','Pnt':'TTimes_Punted','Yds.4':'TPunt_Yds','Y/P.1':'TPunt_Yds_Avg','Year':'Season'},inplace=True)
    team2.rename(columns={'Tm':'Team','Unnamed: 5':'Away_Home','TD':'TOpp_Tot_TD','XPA':'TOpp_XP_Att','XPM':'TOpp_XP_Made',\
                          'Att':'TOpp_FG_Att','Md':'TOpp_FG_Made','Sfty':'TOpp_Sfty','Cmp':'TOpp_Pass_Cmp','Att.1':'TOpp_Pass_Att',\
                          'Cmp%':'TOpp_Pass_Cmp%','Yds':'TOpp_Pass_Yds','TD.1':'TOpp_Pass_TD','Int':'TOpp_Pass_Int','Sk':'TOpp_Sk',\
                          'Yds.1':'TOpp_Sk_Yds','Rate':'TOpp_QB_Rate','Att.2':'TOpp_Rush_Att','Yds.2':'TOpp_Rush_Yds',\
                          'Y/A':'TOpp_Rush_Y/A','TD.2':'TOpp_Rush_TD','Tot':'TOpp_Tot_Yds','TO':'TOpp_Tot_TO',\
                          '1stDOpp':'TOpp_1st_Downs','Rush':'TOpp_1st_by_Rsh','Pass':'TOpp_1st_by_Pass','Pen':'TOpp_1st_by_Pen',\
                          'Opp3DAtt':'TOpp_3rd_Down_Att','Opp3DConv':'TOpp_3rd_Down_Conv','Opp3D%':'TOpp_3rd_Down%',\
                          'Opp4DAtt':'TOpp_4th_Down_Att','Opp4DConv':'TOpp_4th_Down_Conv','Opp4D%':'TOpp_4th_Down%',\
                          'Rush.1':'TMargin_Rush','Pass.1':'TMargin_Pass','Tot.1':'TMargin_TotYds','TO.1':'TTO_TD',\
                          'KR':'TKR_TD','PR':'TPR_TD','IR':'TInt_TD','FR':'TFmb_TD','OR':'TOtherRet_TD',\
                          'RetTD':'TAll_Ret_TD','Q1':'TMar_Thru_Q1','Q2':'TMar_Thru_Q2','Q3':'TMar_Thru_Q3',\
                          'Q1.1':'TScore_Diff_Q1','Q2.1':'TScore_Diff_Q2','Q3.1':'TScore_Diff_Q3',\
                          'Q4':'TScore_Diff_Q4','1stHalf':'TScore_Diff_1stHalf','2ndHalf':'TScore_Diff_2ndHalf','Year':'Season'},inplace=True)
    
    # merge the two dataframes based on team, season, date, time, away/home, opp, week, g#, dat, result, and ot
    team = pd.merge(left=team1,right=team2,\
                     how='outer',\
                     on=['Team','Season','Date','Time','Away_Home','Opp','Week','G#','Day','Result','OT'])
    team.set_index('Team',inplace=True)
    # drop all rows that have headers in them
    team.drop('Tm',inplace=True)
    team.reset_index(inplace=True)

    # create a list of all the column names in the team dataframe
    team_cols = []
    
    for col in team.columns:
        team_cols.append(col)

    # cleaning the away/home column to show away or home instead of @
    team.replace({'Away_Home':{'@':'Away',None:'Home'}},inplace=True)
    team[team_cols[11:]] = team[team_cols[11:]].fillna(value=0)
    # change datatype of the columns
    team[['TPass_Cmp','TPass_Att','T3rd_Down_Att','T3rd_Down_Conv','T4th_Down_Att',\
          'T4th_Down_Conv','TOpp_Pass_Cmp','TOpp_Pass_Att','TOpp_3rd_Down_Att',\
          'TOpp_3rd_Down_Conv','TOpp_4th_Down_Att','TOpp_4th_Down_Conv']] = team[['TPass_Cmp','TPass_Att','T3rd_Down_Att',\
                                                                                  'T3rd_Down_Conv','T4th_Down_Att',\
                                                                                  'T4th_Down_Conv','TOpp_Pass_Cmp',\
                                                                                  'TOpp_Pass_Att','TOpp_3rd_Down_Att',\
                                                                                  'TOpp_3rd_Down_Conv','TOpp_4th_Down_Att',\
                                                                                  'TOpp_4th_Down_Conv']].astype(float)
    # calculating the columns to show apporpriate values
    team['TPass_Cmp%'] = team['TPass_Cmp']/team['TPass_Att']
    team['T3rd_Down%'] = team['T3rd_Down_Conv']/team['T3rd_Down_Att']
    team['T4th_Down%'] = team['T4th_Down_Conv']/team['T4th_Down_Att']
    team['TOpp_Pass_Cmp%'] = team['TOpp_Pass_Cmp']/team['TOpp_Pass_Att']
    team['TOpp_3rd_Down%'] = team['TOpp_3rd_Down_Conv']/team['TOpp_3rd_Down_Att']
    team['TOpp_4th_Down%'] = team['TOpp_4th_Down_Conv']/team['TOpp_4th_Down_Att']
    # changing dates/time to datetime values
    team['Date'] = pd.to_datetime(team['Date'],errors='coerce',format='%Y-%m-%d')
    team['TGame_Dur'] = team['TGame_Dur']+':00'
    team['TO_ToP'] = '00:'+team['TO_ToP']
    team['TGame_Dur'] = pd.to_timedelta(team['TGame_Dur'],errors='coerce')
    team['TGame_Dur'] = team['TGame_Dur'].dt.total_seconds()
    team['TO_ToP'] = pd.to_timedelta(team['TO_ToP'],errors='coerce')
    team['TO_ToP'] = team['TO_ToP'].dt.total_seconds()
    #changing datatypes of columns
    team[team_cols[11:16]] = team[team_cols[11:16]].astype(float)
    team[team_cols[17]] = team[team_cols[17]].astype(float)
    team[team_cols[19:]] = team[team_cols[19:]].astype(float)
    # adding month column
    team.insert(loc=9,column='Month',value=team['Date'].dt.month)
    # chaning week datatype and removing any week larger than week 17
    team['Week'] = team['Week'].astype(int)
    team = team[team['Week']<=17]
    team['Week'] = team['Week'].astype(str)
    # saving cleaned data to csv
    team.to_csv(f'../data/clean-data/nfl-team-data-{Season}-clean.csv',index=False)
    
    end = time.time()
    print(f'Team data cleaned. {Season}',f'Time to complete: {end-start}',sep='\n')

### 2.2 - Clean Player Weekly Snap

In [ ]:
# def player_snaps_clean(Season):
    
#     start = time.time()
#     snaps = pd.read_csv(f'../data/raw-data/snap-counts{Season}-raw.csv')
#     # drop unnecessary columns
#     snaps.drop(['Fantasy Pts','Pts/100 Snaps','Rush %','Tgt %','Touch %','Util %','Tackle %','Sack %','QB Hit %','Snaps/Gm'],axis=1,inplace=True)
#     # change datatypes and calculate columns approriately
#     snaps['Season'] = snaps['Season'].astype(str)
#     snaps['Snap %'] = snaps['Snap %']/100
#     snaps['Week'] = snaps['Week'].astype(str)
#     # change team names to be consistent
#     snaps.replace({'Team':{'FA':'','GB':'GNB','JAC':'JAX','KC':'KAN','NE':'NWE','NO':'NOR','SF':'SFO','TB':'TAM'}},inplace=True)
#     # cleaning player names to be consistent
#     snaps['Player'] = snaps['Player']+' '
#     snaps.replace({'Player':{' Jr. ':'',' Jr ':'',' Sr. ':'',' Sr ':'',' III ':'',' II ':'',' IV ':'',' V ':''}},regex=True,inplace=True)
#     snaps['Player'] = snaps['Player'].str.strip(' ')
#     snaps['Player'] = snaps['Player'].str.replace('.','')
#     # save cleaned snap data
#     snaps.to_csv(f'../data/clean-data/snap-counts-{Season}-clean.csv',index=False)
#     end = time.time()

#     print(f'Snap data cleaned. {Season}',f'Time to complete: {end-start}',sep='\n')

### 2.3 - Clean NFL Weekly Injury Reports

In [ ]:
def nfl_injury_clean(Season):
    
    start = time.time()
    injury = pd.read_csv(f'../data/raw-data/nfl-injury-report-{Season}-raw.csv',low_memory=False)
    # separate file to help calculate week numbers
    nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')
    # melt columns to break each week into a separate row
    injury = pd.melt(injury,id_vars=['Player','Team','Season'],var_name='Date', value_name='Status')
    # create 2 columns based off a header
    injury[['Date','Opp']] = injury.Date.str.split('vs. ',expand=True)
    injury[['Month','Day']] = injury.Date.str.split('/',expand=True)
    injury[['Status','Injury']] = injury.Status.str.split(":",expand=True)
    # drop all values that are empty after the melting/splitting
    injury.dropna(axis=0,subset=['Status','Injury'],how='all',inplace=True)
    # change datatypes for cleaning
    injury[['Season','Month','Day']] = injury[['Season','Month','Day']].astype(int)
    # combine datat to create a date for the game and change to datetime value
    injury['Date'] = injury['Date']+'/'+(np.where(injury['Month']<=2,injury['Season']+1,injury['Season'])).astype(str)
    injury['Date'] = pd.to_datetime(injury['Date'])
    nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
    # create function to calculate what nfl week the date occured
    def pre_thu(d):
        days_behind = 3 - d.weekday()
        if days_behind > 0:
            days_behind -= 7
        return d + dt.timedelta(days_behind)
    injury['week_start_nfl'] = injury['Date'].apply(pre_thu)
    nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
    # merge injury data with the nfl week data to pull the weeks into current datagrame
    injury = pd.merge(left=injury,right=nfl_weeks,how='left',left_on='week_start_nfl',right_on='Start Date')
    # update team names for consistency
    injury.replace({'Team':\
                       {'crd':'ARI', 'atl':'ATL', 'rav':'BAL', 'buf':'BUF', 'car':'CAR', 'chi':'CHI', 'cin':'CIN',\
                        'cle':'CLE', 'dal':'DAL', 'den':'DEN', 'det':'DET', 'gnb':'GNB','htx':'HOU','clt':'IND',\
                        'jax':'JAX','kan':'KAN','sdg':'LAC','ram':'LAR','mia':'MIA','min':'MIN','nor':'NOR','nwe':'NWE',\
                        'nyg':'NYG','nyj':'NYJ','rai':'OAK','phi':'PHI','pit':'PIT','sea':'SEA','sfo':'SFO','tam':'TAM',\
                        'oti':'TEN','was':'WAS'}},inplace=True)
    # clean injury data for easier management and searching of data
    injury['Injury'] = injury['Injury'].str.strip(' ')
    injury.replace({'Injury':{'right':'','left':'','Right':'','Left':'','Biceps':'Bicep',\
                              'Triceps':'Tricep','Ankles':'Ankle','hip':'Hip','Hips':'Hip','Knees':'Knee',\
                              'Virus':'Illness','Triceps':'Tricep','Oblique':'Abdomen',\
                              'NotInjuryRelated':'Not Injury Related','MedicalIllness':'Illness',\
                              'LowerLeg':'Lower Leg','CoreMuscle':'Abdomen','Abdominal':'Abdomen'}},\
                               regex=True,inplace=True)
    # clean player names for consistency
    injury['Player'] = injury['Player']+' '
    injury.replace({'Player':{' Jr. ':'',' Jr ':'',' Sr. ':'',' Sr ':'',' III ':'',' II ':'',' IV ':'',' V ':''}},regex=True,inplace=True)
    injury['Player'] = injury['Player'].str.strip(' ')
    injury['Player'] = injury['Player'].str.replace('.','')
    # drop unnecessary columns
    injury.drop(['Month','Day','week_start_nfl','Start Date'],axis=1,inplace=True)
    # drop all empty week values
    injury.dropna(subset=['Week'],inplace=True)
    # change datatypes
    injury[['Player','Week','Season']] = injury[['Player','Week','Season']].astype(str)
    # reorganize columns
    injury = injury[['Player','Team','Opp','Date','Season','Week','Status','Injury']]
    # add two empty columns for data input
    injury[['Specific_Inj','Side']] = None
    injury.to_csv(f'../data/clean-data/injury-report-{Season}-clean.csv',index=False)
    end = time.time()
    
    print(f'Injury data cleaned. {Season}',f'Time to complete: {end-start}',sep='\n')

### 2.4 - Clean Player Weekly Stats

In [ ]:
def player_stats_clean(Season):
    
    start = time.time()
    stats = pd.read_csv(f'../data/raw-data/player-stats-{Season}-raw.csv')
    # drop empty rows
    stats.dropna(how='all',inplace=True)
    # drop all rows with header information
    stats.drop(stats[stats['Player'] == 'Player'].index, inplace = True)
    stats.drop('Lg',axis=1,inplace=True)
    # rename columns for better management
    stats.rename(columns={'Tm':'Team','Unnamed: 6':'Away_Home','Cmp':'IPass_Cmp','Att':'IPass_Att','Cmp%':'IPass_Cmp%','Yds':'IPass_Yds',\
                 'TD':'IPass_TD','Int':'IPass_Int','Rate':'IQB_Rate','Sk':'I_Sk','Yds.1':'ISk_Yds','Y/A':'IPass_Y/A',\
                 'AY/A':'IPass_AdjY/A','Att.1':'IRush_Att','Yds.2':'IRush_Yds','Y/A.1':'IRush_Y/A','TD.1':'IRush_TD',\
                 'Tgt':'IRec_Tgt','Rec':'IRec_Rec','Yds.3':'IRec_Yds','Y/R':'IRec_Y/R','TD.2':'IRec_TD','Ctch%':'IRec_Ctch%',\
                 'Y/Tgt':'IRec_Y/Tgt','XPM':'IXP_Made','XPA':'IXP_Att','XP%':'IXP%','FGM':'IFG_Made','FGA':'IFG_Att',\
                 'FG%':'IFG%','2PM':'I2pt_Made','Sfty':'ISfty','TD.3':'ITot_TD','Pts':'ITot_Pts','Rt':'IKR_Rt','Yds.4':'IKR_Yds',\
                 'Y/Rt':'IKR_Y/Rt','TD.4':'IKR_TD','Ret':'IPR_Rt','Yds.5':'IPR_Yds','Y/R.1':'IPR_Y/Rt','TD.5':'IPR_TD',\
                 'Sk.1':'ITack_Sk','Solo':'ITack_Solo','Ast':'ITack_Ast','Comb':'ITack_Tot','TFL':'ITack_TFL',\
                 'QBHits':'ITack_QBHits','Int.1':'IDef_Int','Yds.6':'IDef_IntYds','TD.6':'IDef_IntTD','PD':'IDef_PD',\
                 'Fmb':'IFmb_Fmb','FL':'IFmb_Lost','FF':'IFmb_Forced','FR':'IFmb_Recov','Yds.7':'IFmb_Yds','TD.7':'IFmb_TD'},\
                 inplace=True)
    # create a list of the columns
    stats_cols = []
    for col in stats.columns:
        stats_cols.append(col)
    # replace blanks and @ with home and away
    stats.replace({'Away_Home':{'@':'Away',None:'Home'}},inplace=True)
    # change datatypes and calculate data appropriately
    stats[['IPass_Cmp','IPass_Att','IRec_Rec','IRec_Tgt','IXP_Made','IXP_Att','IFG_Made','IFG_Att']] = stats[['IPass_Cmp','IPass_Att','IRec_Rec','IRec_Tgt','IXP_Made','IXP_Att','IFG_Made','IFG_Att']].astype(float)
    stats['IPass_Cmp%'] = stats['IPass_Cmp']/stats['IPass_Att']
    stats['IRec_Ctch%'] = stats['IRec_Rec']/stats['IRec_Tgt']
    stats['IXP%'] = stats['IXP_Made']/stats['IXP_Att']
    stats['IFG%'] = stats['IFG_Made']/stats['IFG_Att']
    # change datatypes
    stats[stats_cols[11:]] = stats[stats_cols[11:]].astype(float)
    stats[stats_cols[11:]] = stats[stats_cols[11:]].fillna(value=0)
    stats['Date'] = pd.to_datetime(stats['Date'],errors='coerce',format='%Y-%m-%d')
    # insert a column for season
    stats.insert(loc=4,column='Season',value=Season)
    stats['Player'] = stats['Player'].astype(str)
    stats['Week'] = stats['Week'].astype(str)
    stats['Season'] = stats['Season'].astype(str)
    # clean player names for consistency
    stats['Player'] = stats['Player']+' '
    stats.replace({'Player':{' Jr. ':'',' Jr ':'',' Sr. ':'',' Sr ':'',' III ':'',' II ':'',' IV ':'',' V ':''}},regex=True,inplace=True)
    stats['Player'] = stats['Player'].str.strip(' ')
    stats['Player'] = stats['Player'].str.replace('.','')
    stats.to_csv(f'../data/clean-data/player-stats-{Season}-clean.csv',index=False)
    end = time.time()
    
    print(f'Player stats data cleaned. {Season}',f'Time to complete: {end-start}',sep='\n')

## Section 3 - Grouped Run Fucntions

### 3.1 - Scrape All Data

In [ ]:
def run_scrape(Start,End):
    # run all web scrape functions at once for multiple season
    start = time.time()
    season = []
    for yr in range(Start,End+1):
        season.append(yr)
    for s in season:
        team_data_scrape(s,1)
        team_data_scrape(s,2)
        injury_reports_scrape(s)
        player_stats_scape(s)
    end = time.time()
    
    print(f'Scrape complete.',f'Time to complete season {s}: {end-start}',sep='\n')

### 3.2 - Clean All Data

In [ ]:
def run_clean(Start,End):
    # run all data clean functions at once for multiple seasons
    start = time.time()
    season = []
    for yr in range(Start,End+1):
        season.append(yr)
    for s in season:
        team_data_clean(s)
        nfl_injury_clean(s)
        player_stats_clean(s)
    end = time.time()
    
    print(f'Data clean complete.',f'Time to compelte season {s}: {end-start}',sep='\n')

In [ ]:
# run_scrape(2016,2020)

In [ ]:
# run_clean(2016,2020)

In [ ]:
# player_snap_scrape(2016,2020,"y")

## Section 4 - Prepare for Analysis

### 4.1 - Combine all files of one season into a single file

In [43]:
# combine data
def combine_season(Season):

    # open csv files for combining data
    inj = pd.read_csv(f'../data/clean-data/injury-report-{Season}-clean.csv')
    stat = pd.read_csv(f'../data/clean-data/player-stats-{Season}-clean.csv')
    snp = pd.read_csv(f'../data/clean-data/snap-counts-{Season}-clean.csv')
    tm = pd.read_csv(f'../data/clean-data/nfl-team-data-{Season}-clean.csv')
    
    # combine stat and snap data
    ss = pd.merge(left=stat,right=snp,how='outer',on=['Player','Season','Week'])
    ss['Team_x'] = ss['Team_x'].fillna(ss['Team_y'])
    ss['Pos_x'] = ss['Pos_y'].fillna(ss['Pos_x'])
    ss.drop(['Team_y','Pos_y'], axis=1, inplace=True)
    ss.rename(columns={'Team_x':'Team','Pos_x':'Pos'},inplace=True)
    
    # combine stat_snap data and injury data
    ssj = pd.merge(left=ss,right=inj,how='outer',on=['Player','Season','Week'])
    ssj['Date_x'] = ssj['Date_x'].fillna(ssj['Date_y'])
    ssj['Team_x'] = ssj['Team_x'].fillna(ssj['Team_y'])
    ssj['Opp_x'] = ssj['Opp_x'].fillna(ssj['Opp_y'])
    ssj.drop(['Date_y','Team_y','Opp_y','Games'],axis=1,inplace=True)
    ssj.rename(columns={'Date_x':'Date','Team_x':'Team','Opp_x':'Opp'},inplace=True)
    ssj['Player'] = ssj['Player'].str.replace('.','')
    ssj.dropna(subset=['Player'],inplace=True)
    ssj['Week'] = ssj['Week'].astype(int)
    player = ssj['Player'].unique().tolist()
    player = player*17
    player.sort()
    df2 = pd.DataFrame(player,columns=['Player'])
    df2['Season'] = Season
    df2['Week'] = df2.groupby('Player').cumcount()+1
    ssj = pd.merge(left=ssj, right=df2, how='outer', left_on=['Player', 'Season', 'Week'], right_on=['Player', 'Season', 'Week'])

    # combine team data into the individual data to fill in some blanks
    team = tm['Team'].unique().tolist()
    team = team*17
    team.sort()
    team_df = pd.DataFrame(team, columns=['Team'])
    team_df['Season'] = Season
    team_df['Opp'] = "Bye"
    team_df['Week'] = team_df.groupby('Team').cumcount()+1
    tm = pd.merge(left=tm, right=team_df, how="outer", on=['Team', 'Season', 'Week'])
    tm['Opp_x'] = tm['Opp_x'].fillna(tm['Opp_y'])
    tm.rename(columns={'Opp_x':'Opp'}, inplace=True)
    tm.drop('Opp_y', axis=1, inplace=True)
    
"""
Not merging properly because there is no "Team" value in "Team" column when merging below
"""

    ssjt = pd.merge(left=ssj, right=tm, how='outer', left_on=['Team', 'Week'], right_on=['Team', 'Week'])
    return ssjt.sort_values(['Player', 'Week']).head(100)
#     ssjt['Date_x'] = ssjt['Date_x'].fillna(ssjt['Date_y'])
#     ssjt['Away_Home_x'] = ssjt['Away_Home_x'].fillna(ssjt['Away_Home_y'])
#     ssjt['Opp_x'] = ssjt['Opp_x'].fillna(ssjt['Opp_y'])
#     ssjt['G#_x'] = ssjt['G#_x'].fillna(ssjt['G#_y'])
#     ssjt['Day_x'] = ssjt['Day_x'].fillna(ssjt['Day_y'])
#     ssjt['Result_x'] = ssjt['Result_x'].fillna(ssjt['Result_y'])
#     ssjt.drop(['Date_y','Away_Home_y','Opp_y','G#_y','Day_y','Result_y'],axis=1,inplace=True)
#     ssjt.rename(columns={'Date_x':'Date','Away_Home_x':'Away_Home','Opp_x':'Opp','G#_x':'G#','Day_x':'Day','Result_x':'Result'},inplace=True)
    
#      # sort columns for easier viewing
#     ssjt.sort_values(by=['Player','Week'],inplace=True)
    
#     # give/change all player positions to players with positions listed for the current year
#     ssjt.replace({'Pos':{'0':None}},inplace=True)
#     key = ssjt[['Player','Pos']]
#     key = key.drop_duplicates()
#     key = key.dropna()
#     key.replace({'Pos':{'DT':'DL','CB':'DB','FB':'RB','OLB':'LB','S':'DB','G':'OL','DE':'DL','FS':'DB','RT':'OL','T':'OL','RG':'OL','LT':'OL','C':'OL','LS':'LongSnap','LDE':'DL','RILB':'LB','LCB':'DB','ILB':'LB',\
#                        'MLB':'LB','TB':'RB','LDT':'DL','NT':'DL','WILL':'LB','SS':'DB','RDE':'DL','SAM':'LB','HB':'RB','WR/RS':'WR','G/T':'OL','C/G':'OL','G/C':'OL','OT':'OL','CB/RS':'DB','DB/LB':'LB','T/G':'OL',\
#                         'RB/WR':'RB','LB/FB':'LB','DE/LB':'LB','DT/DE':'DL','TE/DE':'TE','K/P':'K','OS':'OL','LG':'OL','MIKE':'LB','LILB':'LB','WLB':'LB','G/OT':'OL','SLB':'LB','RCB':'DB'}},inplace=True)
#     ssjt = pd.merge(ssjt,key,how='left',left_on='Player',right_on='Player')
#     ssjt['Pos_x'] = ssjt['Pos_y']
#     ssjt.drop(['Pos_y'],axis=1,inplace=True)
#     ssjt.rename(columns={'Pos_x':'Pos'},inplace=True)
#     ssjt['Injury'] = ssjt['Injury'].str.lower()
#     ssjt['Injury'] = ssjt['Injury'].str.strip()
#     ssjt.replace({'Injury':
#                          {'biceps':'bicep','abdominal':'abdomen','core':'abdomen',"broken fibula":'fibula','fractured forearm':'forearm',\
#                           'general medical issue':'illness','quadriceps':'quadricep','ribs':'rib','rib cage':'rib','glute':'hip',\
#                           'sprained shoulder':'sprained ac joint','thumb sprain':'thumb','nack':'neck','nack, groin':'neck, groin',\
#                           'torn acl':'acl','torn mcl':'mcl','oblique':'abdomen'}},inplace=True)

    # write data frame to csv
#     ssjt.to_csv(f'../data/clean-data/nfl-{Season}.csv',index=False)

def combine_all_season(Start,End):
    start = time.time()
    for yr in range(Start,End+1):
        combine_season(yr)
    end = time.time()
    print(f'Data clean combined and cleaned further.',f'Time to complete: {end-start}',sep='\n')

In [44]:
# combine_all_season(2016, 2020)
combine_season(2020)

<ipython-input-43-1dbae51fe9f8>:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ssj['Player'] = ssj['Player'].str.replace('.','')


,Player,Pos,Age,Date_x,Season_x,Team,Away_Home_x,Opp_x,Result_x,G#_x,Week,Day_x,IPass_Cmp,IPass_Att,IPass_Cmp%,IPass_Yds,IPass_TD,IPass_Int,IQB_Rate,I_Sk,ISk_Yds,IPass_Y/A,IPass_AdjY/A,IRush_Att,IRush_Yds,IRush_Y/A,IRush_TD,IRec_Tgt,IRec_Rec,IRec_Yds,IRec_Y/R,IRec_TD,IRec_Ctch%,IRec_Y/Tgt,IXP_Made,IXP_Att,IXP%,IFG_Made,IFG_Att,IFG%,I2pt_Made,ISfty,ITot_TD,ITot_Pts,IKR_Rt,IKR_Yds,IKR_Y/Rt,IKR_TD,IPR_Rt,IPR_Yds,IPR_Y/Rt,IPR_TD,ITack_Sk,ITack_Solo,ITack_Ast,ITack_Tot,ITack_TFL,ITack_QBHits,IDef_Int,IDef_IntYds,IDef_IntTD,IDef_PD,IFmb_Fmb,IFmb_Lost,IFmb_Forced,IFmb_Recov,IFmb_Yds,IFmb_TD,Snaps,Snap %,Status,Injury,Specific_Inj,Side,Season_y,Date_y,Time,Away_Home_y,Opp_y,G#_y,Day_y,Month,Result_y,OT,Points_For,Points_Against,PD,Points_Comb,Spread,Vs_Line,Over/Under,OU Result,TPass_Cmp,TPass_Att,TPass_Cmp%,TPass_Yds,TPass_TD,TPass_Int,TSack,TSack_Yds,TQB_Rate,TRush_Att,TRush_Yds,TRush_Y/A,TRush_TD,TTot_Yds,TO_Play#,TO_Y/P,TD_Play#,TD_Y/P,TTot_TO,TO_ToP,TGame_Dur,Pen,TPen_Yds,TOpp_Pen,TOpp_Pen_Yds,TComb_Pen,TComb_Pen_Yds,T1st_Downs,T1st_by_Rsh,T1st_by_Pass,T1st_by_Pen,T3rd_Down_Att,T3rd_Down_Conv,T3rd_Down%,T4th_Down_Att,T4th_Down_Conv,T4th_Down%,TTot_TD,TXP_Att,TXP_Made,TFG_Att,TFG_Made,T2Pt_Att,T2Pt_Made,TSfty,TTimes_Punted,TPunt_Yds,TPunt_Yds_Avg,TOpp_Tot_TD,TOpp_XP_Att,TOpp_XP_Made,TOpp_FG_Att,TOpp_FG_Made,TOpp_Sfty,TOpp_Pass_Cmp,TOpp_Pass_Att,TOpp_Pass_Cmp%,TOpp_Pass_Yds,TOpp_Pass_TD,TOpp_Pass_Int,TOpp_Sk,TOpp_Sk_Yds,TOpp_QB_Rate,TOpp_Rush_Att,TOpp_Rush_Yds,TOpp_Rush_Y/A,TOpp_Rush_TD,TOpp_Tot_Yds,TOpp_Tot_TO,TOpp_1st_Downs,TOpp_1st_by_Rsh,TOpp_1st_by_Pass,TOpp_1st_by_Pen,TOpp_3rd_Down_Att,TOpp_3rd_Down_Conv,TOpp_3rd_Down%,TOpp_4th_Down_Att,TOpp_4th_Down_Conv,TOpp_4th_Down%,TMargin_Rush,TMargin_Pass,TMargin_TotYds,TTO_TD,TKR_TD,TPR_TD,TInt_TD,TFmb_TD,TOtherRet_TD,TAll_Ret_TD,TMar_Thru_Q1,TMar_Thru_Q2,TMar_Thru_Q3,TScore_Diff_Q1,TScore_Diff_Q2,TScore_Diff_Q3,TScore_Diff_Q4,TScore_Diff_1stHalf,TScore_Diff_2ndHalf
1794,A'Shawn Robinson,NaN,NaN,2020-09-13,2020.0,LAR,NaN,DAL,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Injured Reserve,NaN,NaN,NaN,2020.0,2020-09-13,8:20,Home,DAL,1.0,Sun,9.0,W 20-17,NaN,20.0,17.0,3.0,37.0,-1.0,covered,52.0,under,20.0,31.0,0.645161,269.0,0.0,1.0,1.0,6.0,79.4,40.0,153.0,3.83,2.0,422.0,72.0,5.8611,69.0,5.5072,1.0,2138.0,10560.0,5.0,34.0,7.0,50.0,12.0,84.0,23.0,11.0,12.0,0.0,17.0,9.0,0.529412,1.0,1.0,1.000000,2.0,2.0,2.0,3.0,2.0,0.0,0.0,0.0,3.0,142.0,47.33,2.0,2.0,2.0,2.0,1.0,0.0,25.0,39.0,0.641026,244.0,1.0,0.0,3.0,22.0,90.1,27.0,136.0,5.04,1.0,380.0,0.0,24.0,9.0,14.0,1.0,12.0,3.0,0.250000,3.0,1.0,0.333333,17.0,25.0,42.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,-1.0,3.0,7.0,-8.0,4.0,0.0,-1.0,4.0
537,A'Shawn Robinson,NaN,NaN,2020-09-20,2020.0,LAR,NaN,PHI,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Injured Reserve,NaN,NaN,NaN,2020.0,2020-09-20,1:00,Away,PHI,2.0,Sun,9.0,W 37-19,NaN,37.0,19.0,18.0,56.0,2.0,covered,46.0,over,20.0,27.0,0.740741,258.0,3.0,0.0,1.0,9.0,142.1,39.0,191.0,4.90,2.0,449.0,67.0,6.7015,69.0,5.2609,1.0,1873.0,11280.0,4.0,24.0,5.0,30.0,9.0,54.0,30.0,11.0,17.0,2.0,12.0,7.0,0.583333,1.0,1.0,1.000000,5.0,5.0,4.0,1.0,1.0,0.0,0.0,0.0,2.0,96.0,48.00,2.0,1.0,1.0,2.0,2.0,0.0,26.0,43.0,0.604651,242.0,0.0,2.0,0.0,0.0,56.5,26.0,121.0,4.65,2.0,363.0,3.0,25.0,11.0,13.0,1.0,12.0,7.0,0.583333,1.0,0.0,0.000000,70.0,16.0,86.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,5.0,8.0,11.0,-6.0,3.0,10.0,5.0,13.0
18212,A'Shawn Robinson,NaN,NaN,2020-09-27,2020.0,LAR,NaN,BUF,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### 4.2 - Split data after the player sustained an injury for the remainder of the season

In [ ]:
def injured_database(Start,End):
    # create a database of all athletes that sustained an injury and what their production was post injury for the remainder of the season
    start = time.time()
    
    season = []
    for yr in range(Start,End+1):
        season.append(yr)
        
    for s in season:
        df = pd.read_csv(f'../data/clean-data/nfl-{s}.csv',low_memory=False)
        df.sort_values(['Player','Week'],inplace=True)
        df.reset_index(inplace=True)
        df.drop(['index'],axis=1,inplace=True)

        # create list of status's to be able to identify were to split the rows
        status = df['Status'].dropna().unique().tolist()

        # create a database to located the index of the first instance of an injury
        Player = df.loc[df['Status'].isin(status)].drop_duplicates(subset=['Player']).index.unique().tolist()

        # split out the rows then save them to a separate dataframe and export to csv
        inj_df = pd.DataFrame()

        for i in range(0,len(Player)):
            idx = df[df['Player']==df['Player'][Player[i]]].tail(1)
            inj = df.loc[Player[i]-1:idx.index.values.astype(int)[0]]
            inj_df = pd.concat([inj_df,inj])

        inj_df['Inj_Week'] = 0
        inj_df['Inj_Week'] = inj_df.groupby('Player').cumcount()
        inj_df.to_csv(f'../data/analysis-data/injury-db-{s}.csv',index=False)
    end = time.time()
    print(f'Injury databases created for seasons {Start} to {End}.',f'Time to complete: {end-start}',sep='\n')

In [ ]:
# injured_database(2016,2020)

### 4.3 - Split data before the player sustained an injury or for full season healthy players

In [ ]:
def healthy_database(Start,End):
    # create a database of all athletes that were healthy or healthy until sustaining an injury
    start = time.time()
    
    season = []
    for yr in range(Start,End+1):
        season.append(yr)
        
    for s in season:
        df = pd.read_csv(f'../data/clean-data/nfl-{s}.csv',low_memory=False)
        df.sort_values(['Player','Week'],inplace=True)
        df.reset_index(inplace=True)
        df.drop(['index'],axis=1,inplace=True)
        
        # create list of status's to be able to identify were to split the rows
        status = df['Status'].dropna().unique().tolist()
        
        # create a database to locate the index of the first instance of an injury
        Player = df.loc[df['Status'].isin(status)].drop_duplicates(subset=['Player']).index.unique().tolist()
        
        # split out the rows then save them to a separate dataframe and export to csv
        healthy_df = pd.DataFrame()
        
        for i in range(0,len(Player)):
            idx = df[df['Player']==df['Player'][Player[i]]].head(1)
            healthy = df.loc[idx.index.values.astype(int)[0]:Player[i]-1]
            healthy_df = pd.concat([healthy_df,healthy])

        healthy_df['Healthy_Week'] = 0
        healthy_df['Healthy_Week'] = healthy_df.groupby('Player').cumcount()+1
        healthy_df.to_csv(f'../data/analysis-data/healthy-db-{s}.csv',index=False)
    end = time.time()
    print(f'Healthy databases created for seasons {Start} to {End}.',f'Time to complete: {end-start}',sep='\n')

In [ ]:
# healthy_database(2016,2020)

## Section 5 - Dash App

### 5.1 - Create UI From Dash

In [ ]:
def run_app(Start,End):
    inj = pd.DataFrame()
    heal = pd.DataFrame()

    for s in range(Start,End+1):
        i= pd.read_csv(f'../data/analysis-data/injury-db-{s}.csv',low_memory=False)
        h = pd.read_csv(f'../data/analysis-data/healthy-db-{s}.csv',low_memory=False)
        i['Date'] = pd.to_datetime(i['Date'],format='%Y-%m-%d')
        h['Date'] = pd.to_datetime(h['Date'],format='%Y-%m-%d')
        inj = pd.concat([inj,i])
        heal = pd.concat([heal,h])

    inj = inj[['Player','Pos','Team','Season','Fntsy_Pts','Snaps','Status','Injury','Inj_Week']]
    inj = inj.query("Snaps >= 20 and Pos == 'RB'")
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

    inj = pd.pivot_table(inj,index=['Season','Injury'])
    inj.reset_index(inplace=True)
    inj.sort_values("Injury",inplace=True)

    heal = heal[['Player','Pos','Team','Season','Fntsy_Pts','Snaps','Healthy_Week']]
    heal = heal.query("Snaps >= 20 and Pos == 'RB'")
    # df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")

    heal = pd.pivot_table(heal,index=['Season','Player'])
    heal.reset_index(inplace=True)
    heal.sort_values("Fntsy_Pts",ascending=False,inplace=True)

    app = JupyterDash(__name__)
#     app.layout = html.Div([
#             dash_table.DataTable(
#                 id='heal_table',
#                 columns=[{
#                     'name':i,
#                     'id':i,
#                     'type':'numeric',
#                     'format':Format(
#                         scheme=Scheme.fixed,
#                         precision=2,
#                         decimal_delimiter='.',
#                         trim=Trim.yes),
#                     'selectable': True
#                 } for i in heal.columns],
#                 sort_mode = "multi",
#                 sort_action = "native",
#                 data=heal.to_dict('records'),
#                 style_as_list_view = True,
#                 style_cell = {
#                     'padding':'5px',
#                     'minWidth':'95px'
#                 },
#                 style_header = {
#                     'backgroundColor':'#bababa',
#                     'fontWeight':'bold'
#                 },
#                 style_cell_conditional = [
#                     {
#                         'if':{'column_id':c},
#                         'textAlign':'left'
#                     } for c in heal.columns
#                 ],
#                 style_data_conditional = (
#                     [
#                         {
#                             'if':{
#                                 'row_index':'odd'
#                             },
#                             'backgroundColor':'#f5f5f5'
#                         }
#                     ] + 
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{Fntsy_Pts}} = {}'.format(i),
#                                 'column_id' : 'Fntsy_Pts'
#                             },
#                             'backgroundColor':'#14de1b'
#                         }
#                         for i in heal['Fntsy_Pts'].nlargest(5)
#                     ] +
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{{}}} <= {}'.format(col,val),
#                                 'column_id': col
#                             },
#                             'backgroundColor':'red',
#                             'color':'white'
#                         }
#                         for (col,val) in heal.quantile(0.1).iteritems()
#                     ]
#                 ),
#                 fixed_rows = {'headers':True},
#                 fixed_columns = {'headers':True,'data':1},
#                 style_table = {
#                     'height': 300,
#                     'overflowY':'auto',
#                     'overflowX':'auto',
#                     'minWidth':'100%'
#                 },
#                 page_size = 25,
#                 page_action = "native",
#                 page_current = 0,
#             ),
#             html.Div(id='datatable-interactivity-container')])
#     app.layout = html.Div([
#             dash_table.DataTable(
#                 id='inj_table',
#                 columns=[{
#                     'name':i,
#                     'id':i,
#                     'type':'numeric',
#                     'format':Format(
#                         scheme=Scheme.fixed,
#                         precision=2,
#                         decimal_delimiter='.',
#                         trim=Trim.yes),
#                     'selectable': True
#                 } for i in inj.columns],
#                 sort_mode = "multi",
#                 sort_action = "native",
#                 data=inj.to_dict('records'),
#                 style_as_list_view = True,
#                 style_cell = {
#                     'padding':'5px',
#                     'minWidth':'95px'
#                 },
#                 style_header = {
#                     'backgroundColor':'#bababa',
#                     'fontWeight':'bold'
#                 },
#                 style_cell_conditional = [
#                     {
#                         'if':{'column_id':c},
#                         'textAlign':'left'
#                     } for c in inj.columns
#                 ],
#                 style_data_conditional = (
#                     [
#                         {
#                             'if':{
#                                 'row_index':'odd'
#                             },
#                             'backgroundColor':'#f5f5f5'
#                         }
#                     ] + 
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{Fntsy_Pts}} = {}'.format(i),
#                                 'column_id' : 'Fntsy_Pts'
#                             },
#                             'backgroundColor':'#14de1b'
#                         }
#                         for i in inj['Fntsy_Pts'].nlargest(5)
#                     ] +
#                     [
#                         {
#                             'if':{
#                                 'filter_query':'{{{}}} <= {}'.format(col,val),
#                                 'column_id': col
#                             },
#                             'backgroundColor':'red',
#                             'color':'white'
#                         }
#                         for (col,val) in inj.quantile(0.1).iteritems()
#                     ]
#                 ),
#                 fixed_rows = {'headers':True},
#                 fixed_columns = {'headers':True,'data':1},
#                 style_table = {
#                     'height': 300,
#                     'overflowY':'auto',
#                     'overflowX':'auto',
#                     'minWidth':'100%'
#                 },
#                 page_size = 25,
#                 page_action = "native",
#                 page_current = 0,
#             ),
#             html.Div(id='datatable-interactivity-container')])
    app.run_server(mode='inline')#,port=8060)

In [ ]:
# fantasy_socre(2016,2020)

In [ ]:
run_app(2020,2020)

In [ ]:
Start = 2016
End = 2020
inj = pd.DataFrame()
heal = pd.DataFrame()

for s in range(Start,End+1):
    i= pd.read_csv(f'../data/analysis-data/injury-db-{s}.csv',low_memory=False)
    h = pd.read_csv(f'../data/analysis-data/healthy-db-{s}.csv',low_memory=False)
    i['Date'] = pd.to_datetime(i['Date'],format='%Y-%m-%d')
    h['Date'] = pd.to_datetime(h['Date'],format='%Y-%m-%d')
    inj = pd.concat([inj,i])
    heal = pd.concat([heal,h])
inj['Fntsy_Pts'] = ((inj['IPass_Yds'] * (1/25)) +
                        (inj['IPass_TD'] * 4) +
                        (inj['IPass_Int'] * -2) +
                        (inj['IRush_Yds'] * (1/10)) +
                        (inj['IRush_TD'] * 6) +
                        (inj['IRec_Yds'] * (1/10)) +
                        (inj['IRec_TD'] * 6) +
                        (inj['IRec_Rec'] * 0.5) +
                        (inj['I2pt_Made'] * 2) +
                        (inj['IFmb_Lost'] * -2) +
                        (inj['IKR_TD'] * 6) +
                        (inj['IPR_TD'] * 6) +
                        (inj['ITack_Sk'] * 3) +
                        (inj['ITack_Solo'] * 1) +
                        (inj['ITack_Ast'] * 0.5) +
                        (inj['ITack_TFL'] * 1.5) +
                        (inj['IDef_Int'] * 3) +
                        (inj['IFmb_Forced'] * 3) +
                        (inj['IFmb_Recov'] * 3) +
                        ((inj['ITot_TD'] -
                          inj['IPass_TD'] -
                          inj['IRush_TD'] -
                          inj['IRec_TD'] -
                          inj['IKR_TD'] -
                          inj['IPR_TD']) * 6) +
                        (inj['ISfty'] * 2) +
                        (inj['IDef_PD'] * 3))

heal['Fntsy_Pts'] = ((heal['IPass_Yds'] * (1/25)) +
                        (heal['IPass_TD'] * 4) +
                        (heal['IPass_Int'] * -2) +
                        (heal['IRush_Yds'] * (1/10)) +
                        (heal['IRush_TD'] * 6) +
                        (heal['IRec_Yds'] * (1/10)) +
                        (heal['IRec_TD'] * 6) +
                        (heal['IRec_Rec'] * 0.5) +
                        (heal['I2pt_Made'] * 2) +
                        (heal['IFmb_Lost'] * -2) +
                        (heal['IKR_TD'] * 6) +
                        (heal['IPR_TD'] * 6) +
                        (heal['ITack_Sk'] * 3) +
                        (heal['ITack_Solo'] * 1) +
                        (heal['ITack_Ast'] * 0.5) +
                        (heal['ITack_TFL'] * 1.5) +
                        (heal['IDef_Int'] * 3) +
                        (heal['IFmb_Forced'] * 3) +
                        (heal['IFmb_Recov'] * 3) +
                        ((heal['ITot_TD'] -
                          heal['IPass_TD'] -
                          heal['IRush_TD'] -
                          heal['IRec_TD'] -
                          heal['IKR_TD'] -
                          heal['IPR_TD']) * 6) +
                        (heal['ISfty'] * 2) +
                        (heal['IDef_PD'] * 3))

data = pd.concat([inj,heal])
data = data[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Status','Injury','Inj_Week','Healthy_Week']]

inj = inj[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Status','Injury','Inj_Week']]
heal = heal[['Player','Pos','Team','Season','Week','Fntsy_Pts','Snaps','Healthy_Week']]

piv_inj = pd.pivot_table(inj,index=['Season','Injury'])
piv_inj.reset_index(inplace=True)
piv_inj.sort_values("Injury",inplace=True)

piv_heal = pd.pivot_table(heal,index=['Season','Player'])
piv_heal.reset_index(inplace=True)
piv_heal.sort_values("Fntsy_Pts",ascending=False,inplace=True)

In [ ]:
data.sort_values('Fntsy_Pts',ascending=False).head(20)